# Data Association of Noisy Stanford Bunny with Outliers

In [ ]:
import time
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation
import clipperpy

In [ ]:
def generate_dataset(pcfile, m, n1, n2o, outrat, sigma, T_21):
    """Generate Dataset
    """

    pcd = o3d.io.read_point_cloud(pcfile)

    n2 = n1 + n2o # number of points in view 2
    noa = round(m * outrat) # number of outlier associations
    nia = m - noa # number of inlier associations

    if nia > n1:
        raise ValueError("Cannot have more inlier associations "
                         "than there are model points. Increase"
                         "the number of points to sample from the"
                         "original point cloud model.")

    # radius of outlier sphere
    R = 1

    # Downsample from the original point cloud, sample randomly
    I = np.random.choice(len(pcd.points), n1, replace=False)
    D1 = np.asarray(pcd.points)[I,:].T

    # Rotate into view 2 using ground truth transformation
    D2 = T_21[0:3,0:3] @ D1 + T_21[0:3,3].reshape(-1,1)

    # Add noise uniformly sampled from a sigma cube around the true point
    eta = np.random.uniform(low=-sigma/2., high=sigma/2., size=D2.shape)

    # Add noise to view 2
    D2 += eta

    def randsphere(m,n,r):
        from scipy.special import gammainc
        X = np.random.randn(m, n)
        s2 = np.sum(X**2, axis=1)
        X = X * np.tile((r*(gammainc(n/2,s2/2)**(1/n)) / np.sqrt(s2)).reshape(-1,1),(1,n))
        return X

    # Add outliers to view 2
    O2 = randsphere(n2o,3,R).T + D2.mean(axis=1).reshape(-1,1)
    D2 = np.hstack((D2,O2))

    # Correct associations to draw from
    Agood = np.tile(np.arange(n1).reshape(-1,1),(1,2))

    # Incorrect association to draw from
    Abad = np.zeros((n1*n2 - n1, 2))
    itr = 0
    for i in range(n1):
        for j in range(n2):
            if i == j:
                continue
            Abad[itr,:] = [i, j]
            itr += 1

    # Sample good and bad associations to satisfy total
    # num of associations with the requested outlier ratio
    IAgood = np.random.choice(Agood.shape[0], nia, replace=False)
    IAbad = np.random.choice(Abad.shape[0], noa, replace=False)
    A = np.concatenate((Agood[IAgood,:],Abad[IAbad,:])).astype(np.int32)

    # Ground truth associations
    Agt = Agood[IAgood,:]
    
    return (D1, D2, Agt, A)

In [ ]:
m = 1000      # total number of associations in problem
n1 = 1000     # number of points used on model (i.e., seen in view 1)
n2o = 250     # number of outliers in data (i.e., seen in view 2)
outrat = 0.9 # outlier ratio of initial association set
sigma = 0.01  # uniform noise [m] range

# generate random (R,t)
T_21 = np.eye(4)
T_21[0:3,0:3] = Rotation.random().as_matrix()
T_21[0:3,3] = np.random.uniform(low=-5, high=5, size=(3,))

pcfile = '../data/bun10k.ply'

D1, D2, Agt, A = generate_dataset(pcfile, m, n1, n2o, outrat, sigma, T_21)

In [ ]:
iparams = clipperpy.invariants.EuclideanDistanceParams()
iparams.sigma = 0.01
iparams.epsilon = 0.02
invariant = clipperpy.invariants.EuclideanDistance(iparams)

params = clipperpy.Params()
params.rounding = clipperpy.Rounding.DSD_HEU
clipper = clipperpy.CLIPPER(invariant, params)

t0 = time.perf_counter()
clipper.score_pairwise_consistency(D1, D2, A)
t1 = time.perf_counter()
print(f"Affinity matrix creation took {t1-t0:.3f} seconds")

t0 = time.perf_counter()
clipper.solve()
t1 = time.perf_counter()

# A = clipper.get_initial_associations()
Ain = clipper.get_selected_associations()

p = np.isin(Ain, Agt)[:,0].sum() / Ain.shape[0]
r = np.isin(Ain, Agt)[:,0].sum() / Agt.shape[0]
print(f"CLIPPER selected {Ain.shape[0]} inliers from {A.shape[0]} "
      f"putative associations (precision {p:.2f}, recall {r:.2f}) in {t1-t0:.3f} s")

In [ ]:
model = o3d.geometry.PointCloud()
model.points = o3d.utility.Vector3dVector(D1.T)
model.paint_uniform_color(np.array([0,0,1.]))
data = o3d.geometry.PointCloud()
data.points = o3d.utility.Vector3dVector(D2.T)
data.paint_uniform_color(np.array([1.,0,0]))

# corr = o3d.geometry.LineSet.create_from_point_cloud_correspondences(model, data, Ain)
# o3d.visualization.draw_geometries([model, data, corr])

In [ ]:
p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
That_21 = p2p.compute_transformation(model, data, o3d.utility.Vector2iVector(Ain))

In [ ]:
def get_err(T, That):
    Terr = np.linalg.inv(T) @ That
    rerr = abs(np.arccos(min(max(((Terr[0:3,0:3]).trace() - 1) / 2, -1.0), 1.0)))
    terr = np.linalg.norm(Terr[0:3,3])
    return (rerr, terr)

In [ ]:
get_err(T_21, That_21)

In [ ]:
def draw_registration_result(source, target, transformation):
    import copy
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [ ]:
draw_registration_result(model, data, That_21)

---

### Custom Invariant Function

For most cases, we recommend using the provided invariants written in C++ for computational efficiency. In particular, for C++ invariant implementations, we use `OpenMP` to parallelize the computation of the affinity matrix.

However, for quick tests and prototyping it can be convenient to test invariants using Python. In this case, you can extend the C++ `clipper.invariants.PairwiseInvariant` class in Python. Note that this method disables the `OpenMP` parallelization is so will be many times slower than a C++ implementation. On average, for the following Python example invariant, the `score_pairwise_consistency` method takes 6 seconds for 1000 initial associations.

In [ ]:
class Custom(clipperpy.invariants.PairwiseInvariant):
    def __init__(self, σ=0.06, ϵ=0.01):
        clipperpy.invariants.PairwiseInvariant.__init__(self)
        self.σ = σ
        self.ϵ = ϵ
        
    def __call__(self, ai, aj, bi, bj):
        l1 = np.linalg.norm(ai - aj)
        l2 = np.linalg.norm(bi - bj)
        
        c = np.abs(l1 - l2)
        
        return np.exp(-0.5*c**2/self.σ**2) if c < self.ϵ else 0

In [ ]:
c = Custom(σ=0.015, ϵ=0.02)

In [ ]:
params = clipperpy.Params()
clipper = clipperpy.CLIPPER(c, params)

t0 = time.perf_counter()
clipper.score_pairwise_consistency(D1, D2, A)
t1 = time.perf_counter()
print(f"Affinity matrix creation took {t1-t0:.3f} seconds")

t0 = time.perf_counter()
clipper.solve()
t1 = time.perf_counter()

# A = clipper.get_initial_associations()
Ain = clipper.get_selected_associations()

p = np.isin(Ain, Agt)[:,0].sum() / Ain.shape[0]
r = np.isin(Ain, Agt)[:,0].sum() / Agt.shape[0]
print(f"CLIPPER selected {Ain.shape[0]} inliers from {A.shape[0]} "
      f"putative associations (precision {p:.2f}, recall {r:.2f}) in {t1-t0:.3f} s")

### Pure Python Implementation of Pairwise Consistency Scoring

In [ ]:
def k2ij(k, n):
    k += 1
    
    l = n * (n-1) / 2 - k
    o = np.floor( (np.sqrt(1 + 8*l) - 1) / 2. )
    p = l - o * (o + 1) / 2
    i = n - (o + 1)
    j = n - p
    
    return int(i-1), int(j-1)

def score_pairwise_consistency(invariant, D1, D2, A):
    if A is None:
        A = clipperpy.utils.create_all_to_all(D1.shape[1], D2.shape[1])
        
    m = A.shape[0]
    
    M = np.eye(m)
    C = np.ones((m,m))
    
    for k in range(int(m*(m-1)/2)):
        i, j = k2ij(k, m) # or clipperpy.utils.k2ij
        
        if A[i,0] == A[j,0] or A[i,1] == A[j,1]:
            C[i,j] = C[j,i] = 0
            continue
            
        d1i = D1[:,A[i,0]]
        d1j = D1[:,A[j,0]]
        
        d2i = D2[:,A[i,1]]
        d2j = D2[:,A[j,1]]
        
        scr = invariant(d1i,d1j,d2i,d2j)
        if scr > 0:
            M[i,j] = M[j,i] = scr
        else:
            C[i,j] = C[j,i] = 0
    
    return M, C

In [ ]:
t0 = time.time()
_, _ = score_pairwise_consistency(c, D1, D2, A.astype('int'))
t1 = time.time()
print(f"Affinity matrix creation took {t1-t0:.3f} seconds")